In [1]:
import subprocess
import pandas as pd
import os
import glob
from tqdm import tqdm
from os.path import basename as bn, join, split as sp
import parselmouth
import numpy as np
from parselmouth.praat import call
from scipy.io.wavfile import write
import praat_formants_python as pfp


# DATASET Path and Constants

In [2]:
# Write paths
ALL_EXP_FOLDER = "./exports/"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(ALL_EXP_FOLDER) #make export folder

# Synth Vowel info Export CSV filename
SYNTH_VOWEL_INFO_IMP_FILENAME = "d_base_formants_synth_vowels_vowlimLIM.csv"
SYNTH_VOWEL_INFO_IMP_FILEPATH = join(ALL_EXP_FOLDER, SYNTH_VOWEL_INFO_IMP_FILENAME)

# Vowel subset Export CSV filename
SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME = "e_synth-vowels_formant_estimation_vowlimLIM.csv"
SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH = join(ALL_EXP_FOLDER, SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME)


### Import Synth Vowel (BASE) Formant Info dataframe

In [3]:
VOWEL_LIMIT = 100
SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF = pd.read_csv(SYNTH_VOWEL_INFO_IMP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}"))
SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF

,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,F2_median_praat_base,F3_median_praat_base,synth_vowel_path
0,0,MWGR0,M,0.066062,iy,118.56,495.03,2131.18,2950.80,495.03,2124.55,2957.00,./audio_exports/vowlim100/iy_0_MWGR0_M_119.wav
1,1,MJES0,M,0.074375,iy,112.66,316.93,2098.74,2844.58,325.92,2098.60,2876.64,./audio_exports/vowlim100/iy_1_MJES0_M_113.wav
2,2,MSTK0,M,0.077688,iy,150.65,426.11,1931.06,2447.48,420.31,2046.93,2372.78,./audio_exports/vowlim100/iy_2_MSTK0_M_151.wav
3,3,MJDM0,M,0.121500,iy,94.78,467.55,1824.06,2588.08,450.22,1824.37,2533.09,./audio_exports/vowlim100/iy_3_MJDM0_M_95.wav
4,4,MLBC0,M,0.086125,iy,107.83,460.67,1942.34,2558.01,461.12,2026.29,2558.57,./audio_exports/vowlim100/iy_4_MLBC0_M_108.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816,3816,MRJM3,M,0.085500,ax-h,112.14,743.40,1492.04,2672.56,671.51,1352.75,2550.95,./audio_exports/vowlim100/ax-h_3816_MRJM3_M_11...
3817,3817,MAPV0,M,0.070187,ax-h,133.61,646.22,1847.19,2720.36,613.69,1837.41,2696.50,./audio_exports/vowlim100/ax-h_3817_MAPV0_M_13...
3818,3818,MSFV0,M,0.174937,ax-h,99.52,765.19,1876.51,3195.29,600.50,1647.24,2806.36,./audio_exports/vowlim100/ax-h_3818_MSFV0_M_10...
3819,3819,FLJG0,F,0.073313,ax-h,195.82,852.51,1813.99,2963.22,861.90,1809.55,2976.29,./audio_exports/vowlim100/ax-h_3819_FLJG0_F_19...


## FUNCTION: Measure formants of synth vowels | PRAAT FORMANTS

In [4]:
def measure_pitch(audio_path: str) -> float:
    f0min, f0max = [75, 600]
    
    sound = parselmouth.Sound(audio_path) # read the sound
    pitch = call(sound, "To Pitch", 0, f0min, f0max) #create a praat pitch object
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz") # get mean pitch
    return mean_pitch
    
def measure_formants_pfp(audio_path: str, start_sec: float, end_sec: float) -> dict:
    formants: list = pfp.formants_at_interval(
        audio_path, start_sec, end_sec, maxformant=5500, winlen=0.025, preemph=50
    )

    pitch_mean = measure_pitch(audio_path)
    pitch_mean = np.round(pitch_mean, 2)
    
    formants_mean = formants.mean(axis=0)
    formants_mean = list(formants_mean)[1:]  # skip time
    formants_mean = np.round(formants_mean, 2)  # round

    formants_median = np.median(formants, axis=0)
    formants_median = list(formants_median)[1:]  # skip time
    formants_median = np.round(formants_median, 2) # round


    return {
        "pitch_mean_synthvow_praat": pitch_mean,

        "F1_mean_synthvow_praat": formants_mean[0],
        "F2_mean_synthvow_praat": formants_mean[1],
        "F3_mean_synthvow_praat": formants_mean[2],

        "F1_median_synthvow_praat": formants_median[0],
        "F2_median_synthvow_praat": formants_median[1],
        "F3_median_synthvow_praat": formants_median[2],
    }


## FUNCTION: Measure formants of synth vowels | DEEPFORMANTS

In [5]:
def get_deepformants(audio_file: str, begin: float, end: float):
    proc = subprocess.check_output('/home/jeevan/dev/anaconda3/envs/pytorch/bin/python formants.py ' + audio_file + ' /home/jeevan/Desktop/DeepFormants/test.csv --begin ' +  str(begin) + ' --end ' + str(end), cwd="/home/jeevan/Desktop/DeepFormants/", shell=True)
    df = pd.read_csv("/home/jeevan/Desktop/DeepFormants/test.csv")
    return df.iloc[0,1:].tolist()
   

def measure_formants_df(audio_path: str,  start_sec: float, end_sec: float):
    _formants: list = get_deepformants(audio_path, start_sec, end_sec)
    _formants = np.round(_formants, 2)  # round

    return {
        "F1_mean_synthvow_deepformant": _formants[0],
        "F2_mean_synthvow_deepformant": _formants[1],
        "F3_mean_synthvow_deepformant": _formants[2],
        "F4_mean_synthvow_deepformant": _formants[3],
    }


### FUNCTION TESTS

In [8]:
v, audf = SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF.loc[np.random.randint(0, len(SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF)), ["vowel_name", "synth_vowel_path"]]
dur = 1.7/2
start, end = (dur - 0.5), (dur + 0.5)
audf = os.path.realpath(audf)
print(audf, start, end )

print(measure_formants_pfp(audf, start, end))
print(measure_formants_df(audf, start, end))


/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/audio_exports/vowlim100/ah_3348_FHXS0_F_183.wav 0.35 1.35
{'pitch_mean_synthvow_praat': 183.12, 'F1_mean_synthvow_praat': 552.46, 'F2_mean_synthvow_praat': 1085.94, 'F3_mean_synthvow_praat': 1676.75, 'F1_median_synthvow_praat': 552.58, 'F2_median_synthvow_praat': 1085.97, 'F3_median_synthvow_praat': 1676.33}
{'F1_mean_synthvow_deepformant': 497.55, 'F2_mean_synthvow_deepformant': 1639.78, 'F3_mean_synthvow_deepformant': 2648.5, 'F4_mean_synthvow_deepformant': 4019.14}


## FUNCTION: Synth Vowel PITCH, FORMANT Estimation | Praat, DeepFormants, +

In [9]:
def estimate_synth_vowel_formants(vowel_info):
    audio_file = vowel_info["synth_vowel_path"]
    audio_file = os.path.realpath(audio_file)
    vowel_name = vowel_info["vowel_name"]

    mdur = 1.7/2
    start_sec, end_sec = (mdur - 0.5), (mdur + 0.5)

    formant_estimates_pfp = measure_formants_pfp(audio_file, start_sec, end_sec) # estimator 1
    formant_estimates_df = measure_formants_df(audio_file, start_sec, end_sec) # estimator 2
    # TODO: estimators 3, 4, 5

    new_vowel_info = dict(vowel_info) | formant_estimates_pfp | formant_estimates_df

    return new_vowel_info

syvinfo = SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF.loc[np.random.randint(0, len(SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF))]
estimate_synth_vowel_formants(syvinfo)

{'index': 911,
 'person_id': 'FDAC1',
 'sex': 'F',
 'duration_second': 0.099625,
 'vowel_name': 'uw',
 'pitch_mean_praat_base': 164.79,
 'F1_mean_praat_base': 466.99,
 'F2_mean_praat_base': 1507.31,
 'F3_mean_praat_base': 2619.48,
 'F1_median_praat_base': 464.89,
 'F2_median_praat_base': 1445.54,
 'F3_median_praat_base': 2615.48,
 'synth_vowel_path': './audio_exports/vowlim100/uw_911_FDAC1_F_165.wav',
 'pitch_mean_synthvow_praat': 164.98,
 'F1_mean_synthvow_praat': 479.85,
 'F2_mean_synthvow_praat': 847.73,
 'F3_mean_synthvow_praat': 1559.34,
 'F1_median_synthvow_praat': 485.74,
 'F2_median_synthvow_praat': 853.97,
 'F3_median_synthvow_praat': 1500.34,
 'F1_mean_synthvow_deepformant': 459.53,
 'F2_mean_synthvow_deepformant': 1506.23,
 'F3_mean_synthvow_deepformant': 2560.61,
 'F4_mean_synthvow_deepformant': 3953.87}

### Create and export Synth Vowel Formant Estimation Result dataframe

In [10]:
SYNTH_VOWELS_FORMANT_DF = pd.DataFrame([estimate_synth_vowel_formants(syv_i) for _, syv_i in tqdm(SYNTH_VOWEL_AUDIO_FORMANT_INFO_DF[0:].iterrows())])

csv_path = SYNTH_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}")
print(csv_path)
if not os.path.exists(csv_path):
    SYNTH_VOWELS_FORMANT_DF.to_csv(csv_path, index=False)

SYNTH_VOWELS_FORMANT_DF

3821it [1:37:11,  1.53s/it]

./exports/e_synth-vowels_formant_estimation_vowlim100.csv


,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,...,F1_mean_synthvow_praat,F2_mean_synthvow_praat,F3_mean_synthvow_praat,F1_median_synthvow_praat,F2_median_synthvow_praat,F3_median_synthvow_praat,F1_mean_synthvow_deepformant,F2_mean_synthvow_deepformant,F3_mean_synthvow_deepformant,F4_mean_synthvow_deepformant
0,0,MWGR0,M,0.066062,iy,118.56,495.03,2131.18,2950.80,495.03,...,477.77,964.17,2129.27,477.73,968.30,2129.67,476.76,1877.08,2744.39,4065.85
1,1,MJES0,M,0.074375,iy,112.66,316.93,2098.74,2844.58,325.92,...,326.33,895.63,2099.49,326.27,897.32,2094.84,407.13,1800.39,2574.27,3895.27
2,2,MSTK0,M,0.077688,iy,150.65,426.11,1931.06,2447.48,420.31,...,440.83,902.78,1938.58,440.39,910.59,1939.50,444.01,1756.68,2477.15,3927.64
3,3,MJDM0,M,0.121500,iy,94.78,467.55,1824.06,2588.08,450.22,...,460.02,846.94,1839.25,469.01,866.50,1820.84,456.66,1677.85,2489.65,3919.14
4,4,MLBC0,M,0.086125,iy,107.83,460.67,1942.34,2558.01,461.12,...,445.25,933.36,1942.31,445.38,942.06,1942.74,447.07,1746.88,2513.99,3947.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816,3816,MRJM3,M,0.085500,ax-h,112.14,743.40,1492.04,2672.56,671.51,...,741.56,912.03,1495.84,744.00,915.82,1494.80,584.48,1505.16,2527.48,3919.01
3817,3817,MAPV0,M,0.070187,ax-h,133.61,646.22,1847.19,2720.36,613.69,...,652.14,859.65,1851.73,655.59,869.90,1852.97,549.35,1770.26,2609.22,3974.69
3818,3818,MSFV0,M,0.174937,ax-h,99.52,765.19,1876.51,3195.29,600.50,...,762.64,892.74,1880.78,766.19,888.09,1881.88,636.02,1860.75,2884.74,4033.48
3819,3819,FLJG0,F,0.073313,ax-h,195.82,852.51,1813.99,2963.22,861.90,...,802.38,1082.95,1798.05,802.04,1086.73,1797.11,637.38,1798.50,2779.89,4045.44


In [11]:
SYNTH_VOWELS_FORMANT_DF.describe()

,index,duration_second,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,F2_median_praat_base,F3_median_praat_base,pitch_mean_synthvow_praat,F1_mean_synthvow_praat,F2_mean_synthvow_praat,F3_mean_synthvow_praat,F1_median_synthvow_praat,F2_median_synthvow_praat,F3_median_synthvow_praat,F1_mean_synthvow_deepformant,F2_mean_synthvow_deepformant,F3_mean_synthvow_deepformant,F4_mean_synthvow_deepformant
count,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000
mean,1910.000000,0.116880,164.518642,610.393455,1596.547228,2614.085854,609.844020,1586.787592,2604.047092,164.526349,595.354415,968.253224,1645.894331,599.023054,969.342073,1629.912232,524.955883,1588.785198,2531.574666,3954.801795
std,1103.172017,0.045302,44.306388,128.547243,355.117779,324.429094,140.804466,376.711775,350.903810,44.310575,113.789776,108.423864,317.527335,115.255663,120.228222,336.537985,60.622122,161.135142,151.161110,69.097117
min,0.000000,0.064000,82.940000,255.420000,747.820000,1433.480000,259.070000,657.220000,1382.010000,83.000000,249.110000,663.420000,966.190000,248.930000,613.320000,942.970000,396.950000,1276.000000,2028.600000,3520.880000
25%,955.000000,0.081875,124.320000,521.860000,1335.090000,2430.670000,512.520000,1310.600000,2416.270000,124.050000,514.650000,891.720000,1408.490000,516.600000,892.670000,1370.060000,479.120000,1450.640000,2422.630000,3910.900000
50%,1910.000000,0.105000,162.580000,603.160000,1567.440000,2630.730000,600.960000,1556.010000,2616.660000,162.960000,590.540000,957.890000,1610.080000,595.490000,958.910000,1603.230000,520.030000,1554.030000,2529.620000,3954.500000
75%,2865.000000,0.140187,202.300000,686.360000,1822.480000,2825.760000,690.980000,1815.510000,2831.640000,202.040000,667.480000,1033.090000,1841.460000,672.610000,1035.750000,1847.290000,567.090000,1723.090000,2633.130000,4003.050000
max,3820.000000,0.424375,290.680000,1854.180000,2874.720000,3842.220000,2095.860000,2891.460000,4034.360000,291.250000,961.150000,1851.120000,2820.200000,960.660000,2123.960000,2819.980000,709.990000,1999.080000,2978.470000,4149.250000


# TEST

In [35]:
# :)